In [1]:
from __future__ import print_function
import warnings
from os import environ
from PIL import Image

Image.MAX_IMAGE_PIXELS = None
warnings.simplefilter('ignore', Image.DecompressionBombWarning)
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.layers import Dense, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.layers import AveragePooling2D, Input, Flatten, Activation, Dropout, Dense
from keras.optimizers import Adam
from keras.initializers import glorot_normal
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.regularizers import l2
from keras import backend as K
from keras.models import Sequential
from os import path, getcwd
import pandas as pd
import random
import numpy as np
import shutil

Using TensorFlow backend.


In [2]:
def sve_jpg(df):
    for item in list(df.new_filename):
        if not '.jpg' in item:
            return False
    return True

In [3]:
df = pd.read_csv(path.join(getcwd(), 'all_data_info.csv'))
seed = 123

print(df.shape)
df.head()

(103250, 12)


,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [4]:
print('Sve su jpg: ' + str(sve_jpg(df)))

Sve su jpg: True


In [5]:
threshold = 300

x = list(df['artist'].value_counts())
# broj umjetnika koji imaju vise ili jednako od 300 slika
print(len([a for a in x if a >= threshold]))
# len(set(x)) #---> ukupan broj umjetnika

57


In [6]:
# train, validation, test --- 80, 10, 10
num_train = 240
num_val = 30
num_test = num_val
num_samples = num_train + num_val + num_test
b_size = 60

#lista umjetnika koje ćemo promatrati
temp = df['artist'].value_counts()
artists = temp[temp >= threshold].index.tolist()
# print(artists)

num_artists = len(artists)
print('Prepoznajemo ' + str(num_artists) + ' umjetnika')

Prepoznajemo 57 umjetnika


In [7]:
train_dfs = []
val_dfs = []
test_dfs = []

for a in artists:
    # PROVJERI KASNIJE ŠTA JE S NA=TRUE
    tmp = df[df['artist'].str.startswith(a)].sample(n=num_samples, random_state=seed)
    # print(tmp.shape)
    t_df = tmp.sample(n=num_train, random_state=seed)
    rest_df = tmp.loc[~tmp.index.isin(t_df.index)] # uzmi komplement od t_df
    # print(rest_df.shape)
    v_df = rest_df.sample(n=num_val, random_state=seed)
    te_df = rest_df.loc[~rest_df.index.isin(v_df.index)]
    
    train_dfs.append(t_df)
    val_dfs.append(v_df)
    test_dfs.append(te_df)
    
    # ovo se pokrene samo jednom!!
    copyImagesToFiles(a, t_df, v_df, te_df)

train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)
test_df = pd.concat(test_dfs)

print('train tablica\t\t', train_df.shape)
print('validation tablica\t', val_df.shape)
print('test tablica\t\t', test_df.shape)

Prepoznajemo 57 umjetnika
train tablica		 (13680, 12)
validation tablica	 (1710, 12)
test tablica		 (1710, 12)


In [7]:
def center_crop(img, center_crop_size):
    assert img.shape[2] == 3
    centerw, centerh = img.shape[0] // 2, img.shape[1] // 2
    halfw, halfh = center_crop_size[0] // 2, center_crop_size[1] // 2
    return img[centerw-halfw:centerw+halfw, centerh-halfh:centerh+halfh, :]

# https://jkjung-avt.github.io/keras-image-cropping/
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length, random_cropping=True):
    '''
    Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator
    '''
    while True:
        batch_x, batch_y = next(batches)
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            if random_cropping == True:
                batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
            else:
                batch_crops[i] = center_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)


In [9]:
# velicina slika koje dajemo ulaznom sloju mreze
input_shape = (224, 224, 3)
# velicina batch-a
b_size = 30

train_datagen = ImageDataGenerator(
                horizontal_flip=True)

val_datagen = ImageDataGenerator(
                horizontal_flip=True)
test_datagen = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(
                    '../train',
                    batch_size=b_size,
                    class_mode='categorical')
train_generator = train_datagen.standardize(train_generator)
# na slikama iz train skupa radimo crop na slučajnom mjestu
train_crops = crop_generator(train_generator, 224)

validation_generator = val_datagen.flow_from_directory(
                    '../validation',
                    batch_size=b_size,
                    class_mode='categorical')
# na slikama iz validation skupa radimo centralni crop
val_crops = crop_generator(validation_generator, 224, False)


test_generator = test_datagen.flow_from_directory(
                '../test',
                target_size=(224, 224),
                batch_size=b_size)
test_crops = crop_generator(test_generator, 224, False)

Found 13680 images belonging to 57 classes.
Found 1710 images belonging to 57 classes.
Found 1710 images belonging to 57 classes.


Model mreže

In [10]:
# model mreže inspiriran glavnim člankom

model = Sequential()

model.add(Conv2D(32, 
                 kernel_size=3, 
                 strides=2, 
                 padding='same', 
                 input_shape=input_shape,
                 kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, 
                 kernel_size=3, 
                 strides=2, 
                 padding="same", 
                 input_shape=input_shape,
                 kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(4*num_artists, 
                input_shape=(6272,),
                kernel_initializer=glorot_normal()))
model.add(Activation('relu'))
model.add(Dense(num_artists, 
                input_shape=(4*num_artists,),
                activation='softmax',
                kernel_initializer=glorot_normal()))

tbCallBack = TensorBoard(log_dir='./Graph300-5', 
                         histogram_freq=0, 
                         write_graph=True, 
                         write_images=True)

# koristimo adamov optimizator i metrika je točnost
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

# crta tablicu slojeva mreže
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 112, 112, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 32)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
__________

In [11]:
model.load_weights('pokusaj300-20.h5')

In [ ]:
# treniramo mrežu....
model.fit_generator(train_crops,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=10,
                    validation_data=val_crops,
                    validation_steps=STEP_SIZE_VALID,
                    workers=4,
                    callbacks=[tbCallBack])

model.save_weights('prvi_pokusaj300SVE-SLIKE20.h5')

In [89]:
evaluation = model.evaluate_generator(test_crops,
                         steps=test_generator.n//test_generator.batch_size,
                         workers=4,
                         verbose=1)

print(model.metrics_names)
print(evaluation)

57/57 [==============================] - 147s 3s/step
['loss', 'acc']
[2.445510460619341, 0.40175439234365495]


In [14]:
predictions = model.predict_generator(test_crops,
                        steps=test_generator.n//test_generator.batch_size,
                        workers=4,
                        verbose=1)

57/57 [==============================] - 138s 2s/step


In [108]:
batch_x, batch_y = next(test_crops)
testt = next(test_crops)[0]
print(batch_x[0].shape)
print(testt[0].shape)
print(test_generator.batch_size)

batch_xx, batch_yy = next(test_crops)

(224, 224, 3)
(224, 224, 3)
30


In [105]:
testt_preds = np.empty((1, 30), dtype=np.int32)
for i in range(30):
    np.append(testt_preds, model.predict_classes(testt[i].reshape(1, 224, 224, 3), batch_size=1))
print(testt_preds)

[[48 24 55  6 27 55 50 48 30  1 33 34 55 23 41 51 35 56 41 31 16 51 48 42
  49  9 45 50 24 51]]


In [106]:
class_predictions = np.empty((1710, num_artists), dtype=np.int32)
for i in range(test_generator.n//test_generator.batch_size):
    np.append(class_predictions, model.predict_classes(next(test_crops)[0],
                                                      batch_size=test_generator.batch_size,
                                                      verbose=0))

In [109]:
print(class_prediction.shape)
print(class_prediction)

(1710, 57)
[[-3.2360197e+38 -3.1692977e+38 -3.0359589e+38 ... -3.2762097e+38
  -3.0360104e+38 -3.0893874e+38]
 [-3.1560579e+38 -2.7823629e+38 -3.0358548e+38 ...            nan
             nan            nan]
 [           nan            nan            nan ... -2.6222308e+38
  -2.8090525e+38 -2.6756084e+38]
 ...
 [ 4.9601741e-02  1.9295303e+20  6.0479871e-11 ... -1.4668281e+20
  -2.4777512e-11 -1.5103023e-27]
 [ 3.5855388e-12  1.7425780e+11 -3.0542261e+17 ...  9.0141090e+17
  -4.2026519e-14  4.8967550e-08]
 [-8.9553085e-22  9.4243884e-11  6.2324497e-04 ...  2.6758510e-06
   2.8749628e+00  2.5779765e+00]]


In [ ]:
class_proba_predictions = np.empty((1710, num_artists), dtype=np.float64)
for i in range(test_generator.n//test_generator.batch_size):
    np.append(class_proba_predictions, model.proba_classes(next(test_crops)[0],
                                                      batch_size=test_generator.batch_size,
                                                      verbose=0))

In [24]:
print(predictions.shape)
print(predictions)
print(predictions.argmax(axis=-1))
print(predictions.argmax(axis=-1).shape)
print(test_generator.classes.shape)
print(test_generator.classes)
preds = predictions.argmax(axis=-1)

suma = 0
for i in range(len(test_generator.classes)):
    if(preds[i] == test_generator.classes[i]):
        suma += 1
print(suma)

(1710, 57)
[[1.3721391e-06 2.1506427e-03 9.3933784e-05 ... 7.4925078e-06
  5.5478995e-05 1.3561848e-07]
 [5.5451685e-05 4.7822925e-04 2.4916621e-02 ... 2.5983350e-04
  1.9180885e-05 1.7809258e-03]
 [5.6278664e-03 3.9138282e-03 7.2539458e-04 ... 1.3779622e-02
  8.3085283e-04 2.1484074e-04]
 ...
 [8.9523668e-04 1.0923635e-03 3.6614445e-06 ... 7.0001394e-02
  2.0078375e-04 3.1718373e-02]
 [8.5495278e-04 3.4764560e-03 2.0965934e-02 ... 2.0480256e-03
  1.3708125e-04 1.8774582e-03]
 [7.1308306e-05 1.3812854e-06 4.4604791e-03 ... 2.4917863e-05
  6.2907631e-05 1.2956945e-04]]
[18 44  5 ...  3 29  8]
(1710,)
(1710,)
[ 0  0  0 ... 56 56 56]
22


In [26]:

print(preds == test_generator.classes)
LIST = preds == test_generator.classes
print(sum(LIST))
print(sum(LIST)/len(LIST))
print(preds)

[False False False ... False False False]
22
0.012865497076023392
[18 44  5 ...  3 29  8]


In [29]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(test_generator.classes, preds))
print(classification_report(test_generator.classes, preds))

[[0 0 0 ... 1 0 0]
 [0 0 1 ... 1 1 1]
 [0 0 0 ... 0 0 2]
 ...
 [1 1 2 ... 0 0 2]
 [0 2 3 ... 0 1 1]
 [1 0 0 ... 0 0 0]]
             precision    recall  f1-score   support

          0       0.00      0.00      0.00        30
          1       0.00      0.00      0.00        30
          2       0.00      0.00      0.00        30
          3       0.00      0.00      0.00        30
          4       0.00      0.00      0.00        30
          5       0.00      0.00      0.00        30
          6       0.00      0.00      0.00        30
          7       0.00      0.00      0.00        30
          8       0.00      0.00      0.00        30
          9       0.03      0.03      0.03        30
         10       0.03      0.03      0.03        30
         11       0.00      0.00      0.00        30
         12       0.00      0.00      0.00        30
         13       0.00      0.00      0.00        30
         14       0.00      0.00      0.00        30
         15       0.00      0.0

In [24]:
from keras.applicaeput_tensor=None, 
                     input_shape=None, 
                     pooling=None, 
                     classes=num_artists)
sirovi_vgg16.compile(loss='categorical_crossentropy',
                     optimizer=Adam(lr=1e-4),
                     metrics=['accuracy'])
sirovi_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
sirovi_vgg16.fit_generator(train_crops,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=3,
                    validation_data=val_crops,
                    validation_steps=STEP_SIZE_VALID,
                    workers=2)

Epoch 1/3


In [ ]:
import keras
xception = keras.applications.xception.Xception(include_top=True, 
                                                weights=None, 
                                                input_tensor=None, 
                                                input_shape=None, 
                                                pooling=None, 
                                                classes=57)
xception.compile(loss='categorical_crossentropy',
                     optimizer=Adam(lr=1e-4),
                     metrics=['accuracy'])
xception.summary()
xception.fit_generator(train_crops,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=3,
                    validation_data=val_crops,
                    validation_steps=STEP_SIZE_VALID,
                    workers=4)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

Epoch 1/3


In [ ]:
from keras.applications.mobilenet import MobileNet
mobile = MobileNet(input_shape=None, 
                   alpha=1.0, 
                   depth_multiplier=1, 
                   dropout=1e-3, 
                   include_top=True, 
                   weights=None, 
                   input_tensor=None, 
                   pooling=None, 
                   classes=57)
mobile.compile(loss='categorical_crossentropy',
                     optimizer=Adam(lr=1e-4),
                     metrics=['accuracy'])
mobile.summary()
mobile.fit_generator(train_crops,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=3,
                    validation_data=val_crops,
                    validation_steps=STEP_SIZE_VALID,
                    workers=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 226, 226, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 32)      0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 114, 114, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
__________

Epoch 1/3


In [ ]:
from keras.applications.resnet50 import ResNet50
resnet= ResNet50(include_top=True, 
                 weights=None, 
                 input_tensor=None, 
                 input_shape=None, 
                 pooling=None, 
                 classes=57)

resnet.compile(loss='categorical_crossentropy',
                     optimizer=Adam(lr=1e-4),
                     metrics=['accuracy'])
resnet.summary()
resnet.fit_generator(train_crops,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=3,
                    validation_data=val_crops,
                    validation_steps=STEP_SIZE_VALID,
                    workers=2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Epoch 1/3
